# Detecting Objects using the Wallaroo mobilenet Pipeline in a Video Stream

In this tutorial we will explore using the mobilenet pipeline we created in step 3 to run inference on the frames in a Video and then draw the identified object's bounding boxes, classification and classification confidence.

In [1]:
import torch
import pickle
import wallaroo
import os
import numpy as np
import json
import requests
import time
import imutils
import sys
 
from CVDemoUtils import CVDemo



/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wl = wallaroo.Client()

In [3]:
ws = wl.list_workspaces()
for w in ws:
    if w.name() == 'computer-vision-3':
        wl.set_current_workspace(w)

In [4]:
model_name = 'mobilenet'
onnx_model_path = 'models/mobilenet.pt.onnx'
mobilenet_model = wl.upload_model(model_name, onnx_model_path)

In [5]:
deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(1).memory("8Gi").build()

In [6]:
pipeline_name = 'camera-3-pp'
pipeline = wl.build_pipeline(pipeline_name) \
            .add_model_step(mobilenet_model) \
            .deploy(deployment_config = deployment_config)


Waiting for deployment - this will take up to 45s ..... ok


In [7]:
time.sleep(5) # needed to allow the pipeline to settle in.
url = pipeline._deployment._url()
print(url)


http://engine-lb.camera-3-pp-1424:29502/pipelines/camera-3-pp


## Test the pipeline in a video stream

### Detect and Classify Objects in Video using Wallaroo Shadow Deployment

Next we will load each frame in the input-video feedubg ut to the pipeline for inferencing.  Then using the results we will draw a bounding box around each identified object, print its classification, and the model's confidence that the prediction is accurate on the frame and save the frame to an output video.

As we are executing this notice the time it takes to process each frame.  In the next section we will discuss ways to improve this performance.

In [ ]:
cvDemo = CVDemo()

# The size the image will be resized to
width = 640
height = 480

input_video = "videos/amazon-fresh-go.mp4"
#input_video = "videos/camera2.mp4"
output_video = "videos/amazon-fresh-go-inferenced.mp4"
save_frames_path = "images/output"
#input_video = "videos/ww2-warbirds-in-formation.mp4"
#output_video = "videos/ww2-warbirds-in-formation-inferenced.mp4"

config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'save-frames-path' : save_frames_path, # show the input video with the inferenced results drawn on each frame
    'fps' : 15, # Frames per second
    'endpoint-url' : url, # the pipelines rest api endpoint url
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    #'max-frames' : 400, # the # of frames to capture in the output video
    'skip-frames' : 225, # the # of frames to capture in the output video
    'confidence-target' : 0.10, # only display bounding boxes with confidence > provided #
    'color':CVDemo.CYAN, # color to draw bounding boxes and the text in the statistics
    'inference' : 'WALLAROO_SDK', # "ONNX" or "WALLAROO_API" or "WALLAROO_SDK"
    'onnx_model_path' : onnx_model_path,
    'model_name' : model_name,
    'pipeline' : pipeline, # provide this when using inference WALLAROO_SDK 
    'pipeline_name' : pipeline_name,
    'skip-frames-list' :[(440,460),(1400,1450)]
#    'record-start-frame' : 225, # the # of frames to capture in the output video
#    'record-end-frame' : 275, # the # of frames to capture in the output video  
}
cvDemo.DEBUG = False
cvDemo.detectAndClassifyObjectsInVideo(config)
print("We are done.")




    

11-21-2022 00:38:38.393971 Start Time:07:38:38 PM
11-21-2022 00:38:38.394022 Skipping [225] frames
11-21-2022 00:38:38.402092 Video Properties
11-21-2022 00:38:38.402134    video input:videos/amazon-fresh-go.mp4
11-21-2022 00:38:38.402143    video output:videos/amazon-fresh-go-inferenced.mp4
11-21-2022 00:38:38.402166    format:0.0
11-21-2022 00:38:38.402175    fourcc:828601953.0
11-21-2022 00:38:38.402183    mode:0.0
11-21-2022 00:38:38.402202    buffer:0.0
11-21-2022 00:38:38.402209    width:1280.0
11-21-2022 00:38:38.402216    height:720.0
11-21-2022 00:38:38.402224    fps:25.0
11-21-2022 00:38:38.402259    frame size:(640, 630)
11-21-2022 00:38:38.411007 recordStartFrame:0
11-21-2022 00:38:38.411036 recordEndFrame:0
11-21-2022 00:38:40.734876 Frame:225 Read: 0.0021 Inf: 1.5218 Onnx: 0.1299 Draw: 0.0132 Total: 1.5892
11-21-2022 00:38:42.311336 Frame:226 Read: 0.0013 Inf: 1.5172 Onnx: 0.1286 Draw: 0.0133 Total: 1.5763
11-21-2022 00:38:43.929115 Frame:227 Read: 0.0013 Inf: 1.5562 Onnx

### Conclusion

Notice how simple it is to take the mobilenet object detectors rest api endpoint url and use it in a video stream.  Now its your turn.  Upload a video to this notebook and replace the input-video with the path of the uploaded video.  Update the output video accordingly.

See how well it works for you.  Good luck!


In [ ]:
pipeline.undeploy()
#for d in wl.list_deployments():
#    d.undeploy()
    